In [15]:
from openai import OpenAI
openai_client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [3]:
from minsearch import AppendableIndex

## Setup and Prepare Data

In [4]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [5]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
index = AppendableIndex(
    text_fields=["question", "text", "section"]
)

index.fit(documents)

## Agentic RAG

The biggest difference between traditional RAG (fixed process + rigid) and Agentic RAG is that we let the LLM decide if it needs to invoke the tool or not. We achieve this by using function calling capabilities.

In [7]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
    )

    return results

### Function Calling
the reason we call llms "agents". With it, they can invoke any arbitrary function in order to achieve the given goal.

make the LLM use our search function.

In [9]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ.",
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

#### List of tools for function calling

In [13]:
instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
""".strip()

tools = [search_tool]

question = "I just discovered the course. Can I still join?"

chat_messages = [
    {"role": "developer", "content": instructions},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)

response.output[0]

ResponseFunctionToolCall(arguments='{"query":"Can I still join if I just discovered the course?"}', call_id='call_j9YlRYgtCde2U4WWfGXwEeSs', name='search', type='function_call', id='fc_071fa133e9ec96380068f255c7d73c819898b666e0ce2574e6', status='completed')

### Processing Function Calls



In [ ]:
search_results = search(query="join course late")

LLMs are STATELESS, so we need to include conversation history

send back entire conversation history

In [ ]:
call = response.output[0]
chat_messages.append(call)

Invoke the function, and also save the function call results in chat_messages

In [ ]:
search_results = search(query="join course late")
search_results_json = json.dumps(search_results)

call_output = {
    "type": "function_call_output",
    "call_id": call.call_id,
    "output": search_results_json,
}

chat_messages.append(call_output)

Now we are ready to send the results back to the model.

invoke API with function call and function call results w/ previous conversation history

In [ ]:
response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)

In [ ]:
response.output_text

### Adding Explanations

make agent explain its decision-making process:

In [ ]:
instructions = """
You're a course teaching assistant.
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call
""".strip()

### Test

In [ ]:
tools = [search_tool]

question = "I just discovered the course. Can I still join it?"

chat_messages = [
    {"role": "developer", "content": instructions},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model='gpt-40-mini',
    input=chat_messages,
    tools=tools
)

In [ ]:
response.output[0].content[0].text

The second part contains the function call details.

Making these calls manually be executing the notebook cells is not convenient so let's write some code for automating it.

## Agentic Loop

create more flexible function for handling different types of calls

In [ ]:
def make_call(call):
    f_name = call.name
    arguments = json.loads(call.argument)

    if f_name == "search":
        results = search(**arguments)

    # you can add more functions HERE
    else:
        raise ValueError(f"Unknown function {f_name}")
    
    json_results = json.dumps(results)

    return {
        "type": "function_call_output",
        "call_id": call.call_id,
        "output": json_results,
    }

now the loop

In [ ]:
question = "I just discovered the course. can I still join it?"

chat_messages = [
    {"role": "developer", "content": instructions},
    {"role": "user", "content": question}
]

while True:
    response = openai_client.responses.create(
        model="gpt-4o-mini",
        input=chat_messages,
        tools=tools
    )

    has_function_calls = False

    # add response to chat history for LLM's memory
    chat_messages.extend(response.output)

    for entry in response.output:
        if entry.type == "funtion_call":
            print("Function call:")
            print(entry)
            result = make_call(entry)
            print('   ', 'Output:')
            print('   ', result['output'])
            chat_messages.append(result)
            has_function_calls = True
            print()
        
        elif entry.type == "message":
            print("Assistant:")
            print(entry.content[0].text)
            print()

        if not has_function_calls:
            break